In [1]:
import pandas as pd
from sqlite3 import connect
from datetime import datetime as dt
from IPython.display import Image, display

In [2]:
conn = connect('quotation.db')

In [ ]:
conn.close()

In [ ]:
# RMA LIST FROM MASTER PENDING
rma_list = []
q = '''
        SELECT rma FROM new_ml
    '''
try:
    rl = pd.read_sql(q,conn)
#     display(rl)
    for rma in rl['rma']:
        rma_list.append(rma)
except Exception as e:
    
    print(e)
len(rma_list)

In [ ]:
# RMA LIST FROM TRANSFER
q = '''
        SELECT rma,[return] FROM transfers
        
        WHERE (strftime('%Y/%m/%d',[return]) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR [return] IS NULL)
    '''
try:
    rl = pd.read_sql(q,conn)
#     display(rl)
    for rma in rl['RMA']:
        rma_list.append(rma)
except Exception as e:
    print(e)
len(rma_list)

# RMA LIST FROM TRANSFER
q = '''
        SELECT rma,[return] FROM completed
        
        WHERE (strftime('%Y/%m/%d',[return]) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR [return] IS NULL)
    '''
try:
    rl = pd.read_sql(q,conn)
    display(rl)
    for rma in rl['RMA']:
        rma_list.append(rma)
except Exception as e:
    print(e)
len(rma_list)


In [ ]:
'FMSV2023030009' in rma_list

In [3]:
# exfm_web
q = f'''
        SELECT DISTINCT
            c.[rma no.] AS rma,c.customer_name,c.customer_code AS customer,
            CASE
                WHEN e.location IS NULL THEN (SELECT e.location FROM consolidated c LEFT JOIN engineers e ON c.IN_INSPECT_USER_NAME = e.exfm_name)
                ELSE e.location END AS location,
            c.model AS model_d,c.serial_no AS sn,c.[Date Installed] AS install_date,
            c.[WARRANTY_END_DATE] AS warranty_date,
            c.[RECIEVE_DATE] AS receive_date,c.[IN_INSPECT_DATE] AS inspection_date,
            r.[start time] AS start_time,r.[end time] AS end_time,
            CASE
                WHEN c.[repair size] = 'Major' THEN 'Nặng'
                WHEN C.[repair size] = 'Minor' THEN 'Nhẹ'
                WHEN c.[repair size] IS NOT NULL THEN 'Khác' 
                ELSE c.[repair size] END AS repair_size,
            'Other' as issue_part,
            e.ks,c.[other id] AS wr_report,c.REPAIR_STATUS AS change_stt,c.approval,
            CASE
                WHEN c.approval  IN ('Decline','Cancel') THEN 'Chờ xác nhận (quá 3 tháng)'
                WHEN c.[create] < c.warranty_end_date AND c.repair_status ='Inspection' AND c.[other id] IS NULL THEN 'Chờ duyệt Bảo hành'
                WHEN c.[other id] IS NOT NULL AND c.repair_status ='Inspection' THEN 'Chuẩn bị linh kiện'
                ELSE s.vie_status END AS repair_status,
            CASE
            WHEN c.approval  IN ('Decline','Cancel') THEN 9
            ELSE s.stt_score END  AS e_score,
            CASE
                WHEN c.[other id] NOT NULL AND UPPER(c.[other id]) NOT LIKE '%REJECTED%' THEN 0.5
                ELSE 0 END AS w_score,
            
            c.[update time] AS update_time,c.Shipped as return_date

            FROM 
            ((consolidated c LEFT JOIN engineers e ON c.[update user] = e.exfm_code)
            LEFT JOIN status s ON c.[repair_status] = s.repair_status)
            LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.]
        
    '''
try:
    exfm_web = pd.read_sql(q,conn)
#     display(exfm_web)
except Exception as e:
    print(e)


In [4]:
exfm_web

,rma,CUSTOMER_NAME,customer,location,model_d,sn,install_date,warranty_date,receive_date,inspection_date,...,issue_part,ks,wr_report,change_stt,Approval,repair_status,e_score,w_score,update_time,return_date
0,FMSV2023100039,Bernard Clinic,FMSV00392,HCM,EG-760Z,6G403K047,2021-03-02 00:00:00,2021-03-02 00:00:00,2023-10-26 18:41:13,2023-11-06 17:03:32,...,Other,Lê Quang Thông,None,Shipped,Inspection,Đang giao hàng,7.0,0.0,2023-11-10 13:28:46.379000,2023-11-10 13:28:46.379000
1,FMSV2023070039,Hanoi Medical University Hospital,FMSV00079,Hanoi,VP-4450HD,2V567K687,2016-05-18 00:00:00,2017-05-18 00:00:00,2023-07-14 17:06:42,2023-07-21 19:12:48,...,Other,Nguyễn Khắc Thắng,WTY-FFVN-2307024R,Shipped,Approval,Đang giao hàng,7.0,0.5,2023-11-10 19:16:01.432000,2023-11-10 19:16:01.432000
2,FMSV2023110007,HCAP asset,FMSV00293,HCM,EC-600WL,1C692K065,2014-06-11 00:00:00,2015-06-11 00:00:00,2023-11-02 19:10:27,2023-11-09 15:35:32,...,Other,Nguyễn Thái Nguyên,None,Shipped,Approval,Đang giao hàng,7.0,0.0,2023-11-15 11:21:06.563000,2023-11-15 11:21:06.563000
3,FMSV2023100005,Buon Ma Thuot Medical University Hospital,FMSV00411,HCM,EC-720R/I,3C741K158,2021-12-31 00:00:00,2022-12-31 00:00:00,2023-10-04 10:49:19,2023-10-04 10:54:06,...,Other,Lê Quang Thông,None,Shipped,Approval,Đang giao hàng,7.0,0.0,2023-11-15 11:23:23.667000,2023-11-15 11:23:23.667000
4,FMSV2023080084,TRUNG VUONG HOSPITAL,FMSV00018,HCM,EG-530WR,4G361K015,2018-01-26 00:00:00,2019-01-26 00:00:00,2023-08-24 17:04:17,2023-08-24 17:15:38,...,Other,Lê Quang Thông,None,Shipped,Decline,Chờ xác nhận (quá 3 tháng),9.0,0.0,2023-11-15 11:25:58.320000,2023-11-15 11:25:58.320000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,FMSV2023110057,Son Tay General Hospital,FMSV00446,Hanoi,EB-530S,1B083A083,2013-01-01 00:00:00,2014-01-01 00:00:00,2023-11-22 12:49:41.566000,None,...,Other,Nguyễn Khắc Thắng,None,Receive,None,Đang Kiểm Tra,1.0,0.0,2023-11-22 12:49:41.566000,None
362,FMSV2023110060,Soc Son Hospital,FMSV00049,Hanoi,EG-250WR5,SG202A176,2013-11-12 00:00:00,2014-11-12 00:00:00,2023-11-23 17:53:05.623000,None,...,Other,Đức Nguyễn,None,Receive,None,Đang Kiểm Tra,1.0,0.0,2023-11-23 17:53:05.623000,None
363,FMSV2023110061,71 National Hospital,FMSV00386,Hanoi,EG-530WR,9G361K427,2021-02-06 00:00:00,2022-02-06 00:00:00,2023-11-23 17:55:28.214000,None,...,Other,Đức Nguyễn,None,Receive,None,Đang Kiểm Tra,1.0,0.0,2023-11-23 17:55:28.214000,None
364,FMSV2023110062,HCAP asset,FMSV00293,Hanoi,EC-600WL,1C692K073,2014-06-11 00:00:00,2015-06-11 00:00:00,2023-11-23 17:56:55.934000,None,...,Other,Đức Nguyễn,None,Receive,None,Đang Kiểm Tra,1.0,0.0,2023-11-23 17:56:55.934000,None


In [5]:
exfm_web.to_sql('exfm_web',conn,index=False,if_exists = 'replace')

366

In [18]:
# pending_file on consolidated
q = '''
        SELECT DISTINCT
            [rma no.] AS rma_id,customer_name as customer,c.model, serial_no AS sn,
            Territory, s.vie as type_s,
            STRFTIME('%Y-%m-%d',c.in_inspect_date) AS tr_date,
            STRFTIME('%Y-%m-%d',c.[part select date]) AS quotation_date,
            STRFTIME('%Y-%m-%d',c.[authorized_date]) AS confirm_date,
            '' as repair_note,c.repair_status as status,status.vie_status as p_status,status.stt_score as p_score,
            STRFTIME('%Y-%m-%d',c.shipped) as return_date

            FROM ((consolidated c
            LEFT JOIN acc_tbl_exp a ON c.customer_code = a.[no.])
            LEFT JOIN scopes s ON c.model = s.model)
            LEFT JOIN status ON c.repair_status = status.repair_status

    '''
try:
    display(pd.read_sql(q,conn).head())
except Exception as e:
    print(e)

,rma_id,customer,MODEL,sn,Territory,type_s,tr_date,quotation_date,confirm_date,repair_note,status,p_status,p_score,return_date
0,FMSV2023100039,Bernard Clinic,EG-760Z,6G403K047,None,Ống soi dạ dày,2023-11-06,2023-11-09,2023-11-10,,Shipped,Đang giao hàng,7.0,2023-11-10
1,FMSV2023070039,Hanoi Medical University Hospital,VP-4450HD,2V567K687,TRANG THI,Bộ Xử lý,2023-07-21,2023-10-16,2023-10-16,,Shipped,Đang giao hàng,7.0,2023-11-10
2,FMSV2023110007,HCAP asset,EC-600WL,1C692K065,None,Ống soi đại tràng,2023-11-09,2023-11-09,2023-11-09,,Shipped,Đang giao hàng,7.0,2023-11-15
3,FMSV2023100005,Buon Ma Thuot Medical University Hospital,EC-720R/I,3C741K158,None,Ống soi đại tràng,2023-10-04,2023-10-09,2023-10-09,,Shipped,Đang giao hàng,7.0,2023-11-15
4,FMSV2023080084,TRUNG VUONG HOSPITAL,EG-530WR,4G361K015,ETC,Ống soi dạ dày,2023-08-24,2023-10-07,2023-11-13,,Shipped,Đang giao hàng,7.0,2023-11-15


In [20]:
# part_name
q = f'''
        SELECT 
            DISTINCT p.[RMA No.] AS rma,p.PART_NO AS part_no,
            p.Part_description AS part_description,pn.vie as vie_name
            FROM parts p
            LEFT JOIN part_name pn ON p.part_description = pn.part_description

        
        ORDER BY rma
    '''
try:
    parts = pd.read_sql(q,conn)
    display(parts)
except Exception as e:
    print(e)

,rma,part_no,part_description,vie_name
0,FMSV2023030040,898Y201120C,FSB A,Vỏ ống dẫn
1,FMSV2023030040,379N120028_,HEAT CONDUCTION SHEET,Bộ tản nhiệt
2,FMSV2023030040,113Y120085G,LD PCB ASSY,Bo mạch
3,FMSV2023030040,375N130001_,PACKING LG COVER(S),Vòng nhựa
4,FMSV2023030041,342A979721A,BSA-G166A,Bộ phận uốn cong
...,...,...,...,...
2635,FMSV2023110065,110M171001T_,SCREW,Ốc vít
2636,FMSV2023110065,401N200040A,SPLING HOLDING PLATE(1.3MM),Giá đỡ
2637,FMSV2023110065,308N130116_,SPECIAL SCREW,Ốc vít
2638,FMSV2023110065,350N130688_,RAIL COVER,Thanh ray


In [21]:
# r_code
q = f'''
        SELECT r.[rma no.] as rma,pf.r_code as r_code,
        SUBSTRING(pf.r_description,1,50) as r_description,
        pf.vie_name

        FROM r_code pf
        LEFT JOIN repair_code r ON pf.r_code = r.r_code
        
    '''
try:
    r_code = pd.read_sql(q,conn)
    display(r_code)
except Exception as e:
    print(e)

,rma,r_code,r_description,vie_name
0,FMSV2023100007,R001,ANGL.Replace LR Knob,Bộ phận điều khiển
1,FMSV2023080014,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
2,FMSV2023090168,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
3,FMSV2023090234,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
4,None,R003,ANGL. Replace UD Knob,Bộ phận điều khiển
...,...,...,...,...
1186,FMSV2023090103,R006,AWB.Replace O-Ring/One Way Valve,Vòng cao su
1187,FMSV2023090154,R006,AWB.Replace O-Ring/One Way Valve,Vòng cao su
1188,None,R322,PCB. Replace Main CPU PCB Assembly,Thay thế bo mạch chính
1189,None,R263,FUS.Replace Fuse,Thay cầu chì


In [ ]:
today = dt.now().strftime('%y%m%d')
pending_name = f'pending_{today}.xls'
pending_file.to_excel(f'New Web/{pending_name}',index = False)
exfm_name = f'exfm_web_{today}.xls'
exfm_web.to_excel(f'New Web/{exfm_name}',index = False)
parts_name = f'part_name_{today}.xls'
parts.to_excel(f'New Web/{parts_name}',index = False)
r_code_name = f'r_code_{today}.xls'
r_code.to_excel(f'New Web/{r_code_name}',index = False)


In [ ]:
today = dt.now().strftime('%y%m%d')
folder_name = 'New web'
pending_name = f'pending_{today}.xls'
exfm_name = f'exfm_web_{today}.xls'
parts_name = f'part_name_{today}.xls'
r_code_name = f'r_code_{today}.xls'

# self.pending_file.to_excel(f'New Web/{pending_name}',index = False)
# self.exfm_web.to_excel(f'New Web/{exfm_name}',index = False)
# self.parts.to_excel(f'New Web/{parts_name}',index = False)
# self.r_code.to_excel(f'New Web/{r_code_name}',index = False)
writer = pd.ExcelWriter(f'{folder_name}/{pending_name}')
pending_file.to_excel(writer,sheet_name='Sheet1',index=False)
writer = pd.ExcelWriter(f'{folder_name}/{exfm_name}')
exfm_web.to_excel(writer,sheet_name='Sheet1',index=False)
writer = pd.ExcelWriter(f'{folder_name}/{parts_name}')
parts.to_excel(writer,sheet_name='Sheet1',index=False)
writer = pd.ExcelWriter(f'{folder_name}/{r_code_name}')
r_code.to_excel(writer,sheet_name='Sheet1',index=False)


In [ ]:
# exfm_web
q = '''
        SELECT * FROM r_code
    '''
try:
    display(pd.read_sql(q,conn))
except Exception as e:
    print(e)

In [ ]:
from sqlite3 import connect
conn = connect('history.db')

In [ ]:
import sources.new_web as nw

In [ ]:
run_all = nw.data_process(conn)
run_all.rma_list()
run_all.exfm_web()
run_all.pending_file()
run_all.parts_name()
run_all.r_code()
run_all.export_files()

In [ ]:
!pip freeze
